# ImageQA

Assesses the image quality of fits files based on a given set of parameters and 'approves' or 'rejects' each image
#### Input files must be in subdirectory named 'inputfiles', output is written to 'approval.out'
Current parameters: number of stars in field, FWHM of stars in field

Todo: Add more parameters


In [1]:
import sewpy
from astropy.io import fits
from os import listdir
import json

In [2]:
files = listdir('./inputfiles/') #extract input files

#define SExtractor parameters
sew = sewpy.SEW(params=['X_IMAGE','Y_IMAGE','FLUX_APER(3)','FWHM_IMAGE','FWHM_WORLD'],
                config={'DETECT_MINAREA':10, 'PHOT_APERTURES':'5, 10, 20'},
                sexpath='sex')

In [3]:
def approval(fitsfile):
    """
    Evaluates a FITS image quality based on a set of parameters,
    obtained from SExtractor, current set is:
    Mean FWHM of objects in field
    Number of starts in field
    :param fitsfile: fits file to be evaluated
    :returns approval: boolean for approval of image
    """
    approval = True #initialise boolean
    data = sew(fitsfile) #run SExtractor
    table = data['table'] #extract data
    FWHM_vals = table['FWHM_IMAGE'] #obtain FWHM column
    stars = len(FWHM_vals) #count number of entries
    meanFWHM = sum(FWHM_vals)/stars #evaluate mean FWHM
    
    if stars < 10: #check for stars in field
        approval = False
    if meanFWHM > 8.0: #check for seeing quality
        approval = False

    return approval #output result

In [4]:
approvals = [] #initialise array for booleans
for file in files: #loop through files
    new = approval('./inputfiles/'+file) #evaluate image
    approvals.append(new) #add to array
approvals

Ouch, SExtractor complains :
b''


[True, True, True]

In [5]:
outfile = 'approval.out' #output filename
output = open(outfile,'w') #create file

for i in range(0,len(files)): #loop through files
    new_data = {files[i] : approvals[i]} #create dictionary
    new_json = json.dumps(new_data, ensure_ascii=False) #convert to json
    output.write(new_json+'\n') #write to file
    print(new_data)
output.close()

{'ngc2442_1.fits': True}
{'ngc2442_2.fits': True}
{'ngc2442_3.fits': True}
